In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path

import anndata as ad
import numpy as np
import scanpy as sc
import pandas as pd

In [3]:
DATA_FILENAME = os.path.expanduser('~/data/treeArches/M1_counts.h5ad')
target_species = ['human']

SOURCE_DATA_WRITE_FILENAME = os.path.expanduser('~/data/treeArches/M1_counts_reference.h5ad')
TARGET_DATA_WRITE_FILENAME = os.path.expanduser('~/data/treeArches/M1_counts_query.h5ad')

In [4]:
adata = sc.read(DATA_FILENAME)
adata.var.drop(columns='_index', inplace=True)
adata

AnnData object with n_obs × n_vars = 305638 × 15680
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'Class', 'Subclass', 'RNA_cluster', 'Cross_species', 'species'
    var: 'features'

In [5]:
source_adata = adata[~adata.obs.species.isin(target_species)].copy()
target_adata = adata[adata.obs.species.isin(target_species)].copy()
print(source_adata)
print(target_adata)

AnnData object with n_obs × n_vars = 229017 × 15680
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'Class', 'Subclass', 'RNA_cluster', 'Cross_species', 'species'
    var: 'features'
AnnData object with n_obs × n_vars = 76621 × 15680
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'Class', 'Subclass', 'RNA_cluster', 'Cross_species', 'species'
    var: 'features'


In [6]:
del adata

In [7]:
source_adata.raw = source_adata

sc.pp.normalize_total(source_adata)
sc.pp.log1p(source_adata)

sc.pp.highly_variable_genes(
    source_adata,
    n_top_genes=2000,
    batch_key="species",
    subset=True)

/home/icb/amirali.moinfar/miniconda3/envs/lataq/lib/python3.8/site-packages/scanpy/preprocessing/_highly_variable_genes.py:485: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hvg = hvg.append(missing_hvg, ignore_index=True)
/home/icb/amirali.moinfar/miniconda3/envs/lataq/lib/python3.8/site-packages/scanpy/preprocessing/_highly_variable_genes.py:485: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hvg = hvg.append(missing_hvg, ignore_index=True)


In [8]:
source_adata = ad.AnnData(source_adata.raw[:, source_adata.var_names].X, 
                          obs=source_adata.obs,
                          var=source_adata.var.loc[source_adata.var_names])
target_adata = ad.AnnData(target_adata.raw[:, source_adata.var_names].X,
                          obs=target_adata.obs,
                          var=target_adata.var.loc[source_adata.var_names])

/tmp/ipykernel_1842498/1780026703.py:1: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  source_adata = ad.AnnData(source_adata.raw[:, source_adata.var_names].X,
/tmp/ipykernel_1842498/1780026703.py:4: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  target_adata = ad.AnnData(target_adata.raw[:, source_adata.var_names].X,


In [9]:
source_adata.var

,features,highly_variable,means,dispersions,dispersions_norm,highly_variable_nbatches,highly_variable_intersection
4,ISG15/ISG15/Isg15,True,0.031450,1.041715,1.775931,1,False
20,MXRA8/MXRA8/Mxra8,True,0.075337,0.695115,1.093447,1,False
24,TMEM88B/TMEM88B/Tmem88b,True,0.078610,0.860682,1.436497,2,True
25,VWA1/VWA1/Vwa1,True,0.086747,0.942404,1.585226,1,False
45,HES5/HES5/Hes5,True,0.128222,1.086468,1.808821,2,True
...,...,...,...,...,...,...,...
15534,TFAP2B/TFAP2B/Tfap2b,True,0.010419,0.836281,1.323726,1,False
15550,TNFSF13/TNFSF13/Tnfsf13,True,0.015290,0.458414,0.933954,1,False
15605,FGFBP1/FGFBP1/Fgfbp1,True,0.005051,0.686351,1.377602,1,False
15633,RNASE12/RNASE12/Rnase12,True,0.000278,0.509392,0.921526,1,False


In [10]:
source_adata.write(SOURCE_DATA_WRITE_FILENAME)
target_adata.write(TARGET_DATA_WRITE_FILENAME)